In [1]:
import polars as pl
import pandas as pd
import geopandas as gpd

In [2]:
# odsの中身の確認
ods = pl.read_parquet('/Users/nagampere/File/Python/sayori/od_yonezawa_based_stop_id.parquet/part-00002-tid-7597389897474560305-7a8f602a-b508-4487-af54-55d4c76e5e85-38-1.c000.snappy.parquet')
# ods[2,0] = "53_02"
# ods[2,1] = "56_02"
# ods[8,0] = "52_02"
# ods[8,1] = "86_02"	
# ods[9,0] = "12_02"
# ods[9,1] = "28_02"
ods.head(10)

d_key,o_key,datetime,hour,move_count,weekday_holiday_flag,o_mesh_4,o_stop_name,o_agency,d_mesh_4,d_stop_name,d_agency
str,str,datetime[ns],i32,i32,str,i64,str,str,i64,str,str
"""54_01""","""70_01""",2022-05-02 00:00:00,12,1,"""w""",564070091,"""金池一丁目""","""yonezawa_city""",564060873,"""西部こども園前""","""yonezawa_city"""
"""41_02""","""48_02""",2022-05-13 00:00:00,13,1,"""h""",564060982,"""上杉神社前""","""yonezawa_city""",564060893,"""県営相生ｱﾊﾟｰﾄ東""","""yonezawa_city"""
"""56_02""","""53_02""",2022-05-04 00:00:00,10,1,"""w""",564060874,"""愛宕小前""","""yonezawa_city""",564060973,"""西部ｺﾐｭﾆﾃｨｾﾝﾀｰ前""","""yonezawa_city"""
"""64_01""","""46_02""",2022-05-13 00:00:00,15,1,"""h""",564060884,"""大町一丁目""","""yonezawa_city""",564070083,"""信夫町""","""yonezawa_city"""
"""53_02""","""1_01""",2022-05-30 00:00:00,7,1,"""w""",564061384,"""米沢スキー場前""","""yonezawa_city""",564060874,"""愛宕小前""","""yonezawa_city"""
"""34_01""","""80_02""",2022-05-07 00:00:00,22,1,"""h""",564060693,"""太田町五丁目""","""yonezawa_city""",564070082,"""中央三丁目""","""yonezawa_city"""
"""52_01""","""58_01""",2022-05-26 00:00:00,21,1,"""h""",564070072,"""すこやかセンター""","""yonezawa_city""",564060872,"""二中前""","""yonezawa_city"""
"""12_02""","""27_02""",2022-05-31 00:00:00,17,1,"""w""",564060992,"""米沢営業所""","""yonezawa_city""",564061831,"""万世小学校前""","""yonezawa_city"""
"""86_02""","""52_02""",2022-05-16 00:00:00,16,1,"""w""",564060872,"""二中前""","""yonezawa_city""",564060884,"""九里学園前""","""yonezawa_city"""


In [3]:
import json
ods_parameters = ods.with_columns(
    pl.struct(
        pl.concat_list(pl.col('o_key')).alias("origin_stop_ids"),
        # pl.concat_list(pl.lit("26_01")).alias("origin_stop_ids"),
        pl.concat_list(pl.col('d_key')).alias("destination_stop_ids"),
        # pl.concat_list(pl.lit("42_01")).alias("destination_stop_ids"),
        pl.col('datetime').cast(pl.Date).cast(str).alias("specified_date"),
        # pl.lit("2022-06-20").alias("specified_date"),
        (pl.col('hour')*60*60).cast(int).alias("specified_secs"),
        # (pl.lit('40000')).alias("specified_secs"),
        pl.lit(1).cast(int).alias("transfers_limit"),
        pl.lit(False).alias("is_reverse_search"),
        pl.lit(None).cast(str).alias("available_trip_ids")
    ).alias("parameters")
)
print(ods_parameters[0,12])
ods_parameters.head(10)


{'origin_stop_ids': ['70_01'], 'destination_stop_ids': ['54_01'], 'specified_date': '2022-05-02', 'specified_secs': 43200, 'transfers_limit': 1, 'is_reverse_search': False, 'available_trip_ids': None}


d_key,o_key,datetime,hour,move_count,weekday_holiday_flag,o_mesh_4,o_stop_name,o_agency,d_mesh_4,d_stop_name,d_agency,parameters
str,str,datetime[ns],i32,i32,str,i64,str,str,i64,str,str,struct[7]
"""54_01""","""70_01""",2022-05-02 00:00:00,12,1,"""w""",564070091,"""金池一丁目""","""yonezawa_city""",564060873,"""西部こども園前""","""yonezawa_city""","{[""70_01""],[""54_01""],""2022-05-02"",43200,1,false,null}"
"""41_02""","""48_02""",2022-05-13 00:00:00,13,1,"""h""",564060982,"""上杉神社前""","""yonezawa_city""",564060893,"""県営相生ｱﾊﾟｰﾄ東""","""yonezawa_city""","{[""48_02""],[""41_02""],""2022-05-13"",46800,1,false,null}"
"""56_02""","""53_02""",2022-05-04 00:00:00,10,1,"""w""",564060874,"""愛宕小前""","""yonezawa_city""",564060973,"""西部ｺﾐｭﾆﾃｨｾﾝﾀｰ前""","""yonezawa_city""","{[""53_02""],[""56_02""],""2022-05-04"",36000,1,false,null}"
"""64_01""","""46_02""",2022-05-13 00:00:00,15,1,"""h""",564060884,"""大町一丁目""","""yonezawa_city""",564070083,"""信夫町""","""yonezawa_city""","{[""46_02""],[""64_01""],""2022-05-13"",54000,1,false,null}"
"""53_02""","""1_01""",2022-05-30 00:00:00,7,1,"""w""",564061384,"""米沢スキー場前""","""yonezawa_city""",564060874,"""愛宕小前""","""yonezawa_city""","{[""1_01""],[""53_02""],""2022-05-30"",25200,1,false,null}"
"""34_01""","""80_02""",2022-05-07 00:00:00,22,1,"""h""",564060693,"""太田町五丁目""","""yonezawa_city""",564070082,"""中央三丁目""","""yonezawa_city""","{[""80_02""],[""34_01""],""2022-05-07"",79200,1,false,null}"
"""52_01""","""58_01""",2022-05-26 00:00:00,21,1,"""h""",564070072,"""すこやかセンター""","""yonezawa_city""",564060872,"""二中前""","""yonezawa_city""","{[""58_01""],[""52_01""],""2022-05-26"",75600,1,false,null}"
"""12_02""","""27_02""",2022-05-31 00:00:00,17,1,"""w""",564060992,"""米沢営業所""","""yonezawa_city""",564061831,"""万世小学校前""","""yonezawa_city""","{[""27_02""],[""12_02""],""2022-05-31"",61200,1,false,null}"
"""86_02""","""52_02""",2022-05-16 00:00:00,16,1,"""w""",564060872,"""二中前""","""yonezawa_city""",564060884,"""九里学園前""","""yonezawa_city""","{[""52_02""],[""86_02""],""2022-05-16"",57600,1,false,null}"


In [4]:
# transferデータの修正
stops = pd.read_csv('data_yonezawa/stops.txt')

In [5]:
pdf_stops = gpd.GeoDataFrame(
    stops,
    geometry=gpd.GeoSeries.from_xy(stops['stop_lon'], 
                                   stops['stop_lat'],
                                   crs="EPSG:4326")
).to_crs(6678)

df_stops = pd.DataFrame()
for i, se in pdf_stops.iterrows():
    df_stops = pd.concat([
        df_stops,
        pd.DataFrame({
            "from_stop_id": [se["stop_id"]]*len(pdf_stops),
            "to_stop_id": [x for x in pdf_stops["stop_id"]],
            "distance": pdf_stops["geometry"].apply(lambda x: x.distance(se["geometry"])).tolist()
        })
    ])

df_stops = df_stops.query("distance < 80*3")
df_stops = df_stops.query("from_stop_id != to_stop_id")

In [6]:
transfers = pd.DataFrame()
transfers['from_stop_id'] = df_stops['from_stop_id'].astype(str)
transfers['to_stop_id'] = df_stops['to_stop_id'].astype(str)
transfers['transfer_type'] = 2
transfers['transfer_type'] = transfers['transfer_type'].astype('i4')
transfers['min_transfer_time'] = df_stops['distance'] // 80 * 60
transfers['min_transfer_time'] = transfers['min_transfer_time'].apply(lambda x: -100 if x == 0 else x)
transfers['min_transfer_time'] = transfers['min_transfer_time'].astype('i4')
transfers.reset_index(inplace=True,
                      drop=True)
transfers.to_parquet('data_yonezawa/transfers.parquet')
transfers.to_csv('transfers.csv')
transfers.dtypes

from_stop_id         object
to_stop_id           object
transfer_type         int32
min_transfer_time     int32
dtype: object

In [7]:
display(transfers[transfers["min_transfer_time"]!=1])

,from_stop_id,to_stop_id,transfer_type,min_transfer_time
0,10_01,10_02,2,-100
1,10_02,10_01,2,-100
2,11_01,11_02,2,-100
3,11_02,11_01,2,-100
4,12_01,12_02,2,-100
...,...,...,...,...
285,86_02,86_01,2,-100
286,87_01,31_01,2,120
287,87_01,87_02,2,-100
288,87_02,31_01,2,60


In [8]:
import os
import time
import json

from sayori.raptor import search_p2p_geojson, search_p2p_path
from sayori.models import FeedPath, Feed

In [9]:
def load_data_from_path() -> FeedPath:
    feed_path = {
        "stops": os.environ.get("STOPS_PATH"),
        "trips": os.environ.get("TRIPS_PATH"),
        "stop_times": os.environ.get("STOP_TIMES_PATH"),
        "transfers": os.environ.get("TRANSFERS_PATH"),
        "calendar": os.environ.get("CALENDAR_PATH"),
    }

    return FeedPath.parse_obj(feed_path)

feed_path = load_data_from_path()
feed = Feed.from_feed_path(feed_path)

In [10]:
# pd_ods = ods_parameters.to_pandas()
ods_parameters_res = ods_parameters.head(100).with_columns(
    pl.col("parameters").apply(lambda x:str(search_p2p_geojson(feed,x))).alias("res")
)
# pl.col("parameters").apply(lambda x: x['parameters'])
# req = {
#         "origin_stop_ids": ["26_01"],
#         "destination_stop_ids": ["42_01"],
#         "specified_date": "2022-06-20",
#         "specified_secs": 80000,
#         "transfers_limit": 1,
#         "is_reverse_search": True,
#         "available_trip_ids": None
#     }
res = ods_parameters["parameters"][0]
print(type(ods_parameters["parameters"][0]["origin_stop_ids"]))
print(type(res["origin_stop_ids"]))
res = search_p2p_geojson(feed, res)
print(res)
# test = ods_parameters.with_columns(pl.lit(str(res)).alias('test'))
ods_parameters_res
# .filter(pl.col("res").is_not_null())

<class 'list'>
<class 'list'>
None


d_key,o_key,datetime,hour,move_count,weekday_holiday_flag,o_mesh_4,o_stop_name,o_agency,d_mesh_4,d_stop_name,d_agency,parameters,res
str,str,datetime[ns],i32,i32,str,i64,str,str,i64,str,str,struct[7],str
"""54_01""","""70_01""",2022-05-02 00:00:00,12,1,"""w""",564070091,"""金池一丁目""","""yonezawa_city""",564060873,"""西部こども園前""","""yonezawa_city""","{[""70_01""],[""54_01""],""2022-05-02"",43200,1,false,null}","""None"""
"""41_02""","""48_02""",2022-05-13 00:00:00,13,1,"""h""",564060982,"""上杉神社前""","""yonezawa_city""",564060893,"""県営相生ｱﾊﾟｰﾄ東""","""yonezawa_city""","{[""48_02""],[""41_02""],""2022-05-13"",46800,1,false,null}","""{'type': 'Feat…"
"""56_02""","""53_02""",2022-05-04 00:00:00,10,1,"""w""",564060874,"""愛宕小前""","""yonezawa_city""",564060973,"""西部ｺﾐｭﾆﾃｨｾﾝﾀｰ前""","""yonezawa_city""","{[""53_02""],[""56_02""],""2022-05-04"",36000,1,false,null}","""{'type': 'Feat…"
"""64_01""","""46_02""",2022-05-13 00:00:00,15,1,"""h""",564060884,"""大町一丁目""","""yonezawa_city""",564070083,"""信夫町""","""yonezawa_city""","{[""46_02""],[""64_01""],""2022-05-13"",54000,1,false,null}","""{'type': 'Feat…"
"""53_02""","""1_01""",2022-05-30 00:00:00,7,1,"""w""",564061384,"""米沢スキー場前""","""yonezawa_city""",564060874,"""愛宕小前""","""yonezawa_city""","{[""1_01""],[""53_02""],""2022-05-30"",25200,1,false,null}","""None"""
"""34_01""","""80_02""",2022-05-07 00:00:00,22,1,"""h""",564060693,"""太田町五丁目""","""yonezawa_city""",564070082,"""中央三丁目""","""yonezawa_city""","{[""80_02""],[""34_01""],""2022-05-07"",79200,1,false,null}","""None"""
"""52_01""","""58_01""",2022-05-26 00:00:00,21,1,"""h""",564070072,"""すこやかセンター""","""yonezawa_city""",564060872,"""二中前""","""yonezawa_city""","{[""58_01""],[""52_01""],""2022-05-26"",75600,1,false,null}","""{'type': 'Feat…"
"""12_02""","""27_02""",2022-05-31 00:00:00,17,1,"""w""",564060992,"""米沢営業所""","""yonezawa_city""",564061831,"""万世小学校前""","""yonezawa_city""","{[""27_02""],[""12_02""],""2022-05-31"",61200,1,false,null}","""None"""
"""86_02""","""52_02""",2022-05-16 00:00:00,16,1,"""w""",564060872,"""二中前""","""yonezawa_city""",564060884,"""九里学園前""","""yonezawa_city""","{[""52_02""],[""86_02""],""2022-05-16"",57600,1,false,null}","""{'type': 'Feat…"


In [11]:
print(len(ods_parameters_res))
print(len(ods_parameters_res.filter(pl.col("res")!='None')))

100
41


In [465]:

ods_parameters_res.drop("parameters").write_csv("ods.csv")